### mult & gsm8k

In [1]:
import sys
sys.path.append('../../tools/implicit_chain_of_thought/src')

In [2]:
import math
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
import argparse
import os
import tqdm
import inspect
import logging

import datasets

from models.teacher import Teacher
from models.configuration_teacher import TeacherConfig
from data import CoTDataset, CoTDataCollator, extract_answer

from transformers import AutoTokenizer

/workspace-SR006.nfs2/Bulatov_A/env_main/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')

In [ ]:
class Holder:
    def __init__(self):
        pass

args = Holder()
args.train_path = '/workspace-SR006.nfs2/Bulatov_A/rmt/tools/implicit_chain_of_thought/data/4_by_4_mult/train.txt'
args.val_path = '/workspace-SR006.nfs2/Bulatov_A/rmt/tools/implicit_chain_of_thought/data/4_by_4_mult/valid.txt'
args.batch_size = 8
args.learning_rate = 5e-5

In [26]:
# tokenizer = teacher.tokenizer
collate_fn = CoTDataCollator(tokenizer)
train_dataset = CoTDataset(tokenizer, args.train_path, 1024)
train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, collate_fn=collate_fn, shuffle=True)
val_dataset = CoTDataset(tokenizer, args.val_path, 1024)
val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, collate_fn=collate_fn, shuffle=False)

Creating features from dataset file at /workspace-SR006.nfs2/Bulatov_A/rmt/tools/implicit_chain_of_thought/data/4_by_4_mult/train.txt
tgt_avg:  49.0
src_avg:  10.0
ratios:  0.20408163265306123
tgt_avg:  13.0
src_avg:  10.0
ratios:  0.7692307692307693
 1 3 3 8 * 5 1 0 5 <|endoftext|> 5 5 6 1 4 + 0 1 3 3 8 0 ( 5 6 9 4 2 1 ) + 0 0 0 0 0 0 0 ( 5 6 9 4 2 1 0 ) + 0 0 0 5 5 6 1 4 <|endoftext|> #### 5 6 9 9 7 7 1 4 <|endoftext|>
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 642, 642, 718, 352, 604, 1343, 657, 352, 513, 513, 807, 657, 357, 642, 718, 860, 604, 362, 352, 1267, 1343, 657, 657, 657, 657, 657, 657, 657, 357, 642, 718, 860, 604, 362, 352, 657, 1267, 1343, 657, 657, 657, 642, 642, 718, 352, 604, 220, 50256]
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 1303, 21017, 642, 718, 860, 860, 767, 767, 352, 604, 220, 50256]
[352, 513, 513, 807, 1635, 642, 352, 657, 642, 220, 50256, 1303, 21017, 642, 718, 860, 860, 767, 767, 352, 604, 220, 50256]
 1 3 

In [27]:
# x = val_dataset[0]
# len(x), [i.shape for i in x if isinstance(i, torch.Tensor)]

### convert to hf

In [5]:
def convert_to_hf_format(dataset):
    data_dict = {
        "examples_cot": [],
        "examples_nocot": [],
        "labels_cot": [],
        "labels_cot_shift": [],
        "labels_nocot": [],
        "src_sent_cot": [],
        "src_sent_nocot": [],
        "tgt_sent_cot": [],
        "tgt_sent_nocot": [],
        "examples_only": [],
        "examples_all": [],
        "labels_all": []
    }
    
    for item in dataset:
        (
            examples_cot,
            examples_nocot,
            labels_cot,
            labels_cot_shift,
            labels_nocot,
            src_sent_cot,
            src_sent_nocot,
            tgt_sent_cot,
            tgt_sent_nocot,
            examples_only,
            examples_all,
            labels_all
        ) = item  # Unpack the tuple returned by __getitem__
        
        data_dict["examples_cot"].append(examples_cot.numpy().tolist())
        data_dict["examples_nocot"].append(examples_nocot.numpy().tolist())
        data_dict["labels_cot"].append(labels_cot.numpy().tolist())
        data_dict["labels_cot_shift"].append(labels_cot_shift.numpy().tolist())
        data_dict["labels_nocot"].append(labels_nocot.numpy().tolist())
        data_dict["src_sent_cot"].append(src_sent_cot.numpy().tolist())
        data_dict["src_sent_nocot"].append(src_sent_nocot.numpy().tolist())
        data_dict["tgt_sent_cot"].append(tgt_sent_cot.numpy().tolist())
        data_dict["tgt_sent_nocot"].append(tgt_sent_nocot.numpy().tolist())
        data_dict["examples_only"].append(examples_only.numpy().tolist())
        data_dict["examples_all"].append(examples_all.numpy().tolist())
        data_dict["labels_all"].append(labels_all.numpy().tolist())
    
    return data_dict


train_hf_data_dict = convert_to_hf_format(train_dataset)
hf_train_dataset = datasets.Dataset.from_dict(train_hf_data_dict)

valid_hf_data_dict = convert_to_hf_format(val_dataset)
hf_valid_dataset = datasets.Dataset.from_dict(valid_hf_data_dict)

NameError: name 'train_dataset' is not defined

### 4x4

In [33]:
! du -hs /workspace-SR006.nfs2/Bulatov_A/rmt/data/implicit_chain_of_thought/4_by_4_mult

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2.9G	/workspace-SR006.nfs2/Bulatov_A/rmt/data/implicit_chain_of_thought/4_by_4_mult


In [31]:
save_path = "/workspace-SR006.nfs2/Bulatov_A/rmt/data/implicit_chain_of_thought/4_by_4_mult"
hf_train_dataset.save_to_disk(os.path.join(save_path, "train"))
hf_valid_dataset.save_to_disk(os.path.join(save_path, "valid"))

Saving the dataset (0/7 shards):   0%|          | 0/808000 [00:00<?, ? examples/s]

Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 60091.18 examples/s]


In [ ]:
# print out first all features of sample

for k, v in hf_valid_dataset[0].items():
    tokens = [i if i > 0 else 0 for i in v]
    print(k, tokenizer.decode(tokens))
    print()

examples_cot  5 6 3 2 * 7 4 3 4 <|endoftext|> 5 5 5 6 1 + 0 0 6 4 9 0 ( 5 5 1 1 1 1 ) + 0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 ) + 0 0 0 0 6 4 9 0 <|endoftext|>

examples_nocot  5 6 3 2 * 7 4 3 4 <|endoftext|> #### 5 5 6 0 8 2 0 1 <|endoftext|>

labels_cot !!!!!!!!!!! 5 5 5 6 1 + 0 0 6 4 9 0 ( 5 5 1 1 1 1 ) + 0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 ) + 0 0 0 0 6 4 9 0 <|endoftext|>

labels_cot_shift !!!!!!!!!!<|endoftext|> 5 5 5 6 1 + 0 0 6 4 9 0 ( 5 5 1 1 1 1 ) + 0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 ) + 0 0 0 0 6 4 9 0 <|endoftext|>

labels_nocot !!!!!!!!!!! #### 5 5 6 0 8 2 0 1 <|endoftext|>

src_sent_cot  5 6 3 2 * 7 4 3 4 

src_sent_nocot  5 6 3 2 * 7 4 3 4 

tgt_sent_cot <|endoftext|> 5 5 5 6 1 + 0 0 6 4 9 0 ( 5 5 1 1 1 1 ) + 0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 ) + 0 0 0 0 6 4 9 0 <|endoftext|>

tgt_sent_nocot <|endoftext|> #### 5 5 6 0 8 2 0 1 <|endoftext|>

examples_only  5 6 3 2 * 7 4 3 4 <|endoftext|> 

examples_all  5 6 3 2 * 7 4 3 4 <|endoftext|> 5 5 5 6 1 + 0 0 6 4 9 0 ( 5 5 1 1 1 1 ) + 0 0 5 9 0 7 0 ( 5 

In [ ]:
train_mode = 'cot'

### 5x5

In [ ]:
args.train_path = '/workspace-SR006.nfs2/Bulatov_A/rmt/tools/implicit_chain_of_thought/data/5_by_5_mult/train.txt'
args.val_path = '/workspace-SR006.nfs2/Bulatov_A/rmt/tools/implicit_chain_of_thought/data/5_by_5_mult/valid.txt'

# tokenizer = teacher.tokenizer
collate_fn = CoTDataCollator(tokenizer)
train_dataset = CoTDataset(tokenizer, args.train_path, 1024)
val_dataset = CoTDataset(tokenizer, args.val_path, 1024)

train_hf_data_dict = convert_to_hf_format(train_dataset)
hf_dataset = datasets.Dataset.from_dict(train_hf_data_dict)

valid_hf_data_dict = convert_to_hf_format(val_dataset)
hf_valid_dataset = datasets.Dataset.from_dict(valid_hf_data_dict)

save_path = "/workspace-SR006.nfs2/Bulatov_A/rmt/tools/implicit_chain_of_thought/data/5_by_5_mult/"
hf_train_dataset.save_to_disk(os.path.join(save_path, "train"))
hf_valid_dataset.save_to_disk(os.path.join(save_path, "valid"))

In [ ]:
len(hf_valid_dataset)

1000

### gsm8k

In [9]:
args.train_path = '/workspace-SR006.nfs2/Bulatov_A/rmt/tools/implicit_chain_of_thought/data/gsm8k/train.txt'
args.val_path = '/workspace-SR006.nfs2/Bulatov_A/rmt/tools/implicit_chain_of_thought/data/gsm8k/valid.txt'
args.train_no_aug_path = '/workspace-SR006.nfs2/Bulatov_A/rmt/tools/implicit_chain_of_thought/data/gsm8k/train_no_aug.txt'
args.test_path = '/workspace-SR006.nfs2/Bulatov_A/rmt/tools/implicit_chain_of_thought/data/gsm8k/test.txt'

In [10]:
collate_fn = CoTDataCollator(tokenizer)
val_dataset = CoTDataset(tokenizer, args.val_path, 1024)
test_dataset = CoTDataset(tokenizer, args.test_path, 1024)
train_dataset = CoTDataset(tokenizer, args.train_path, 1024)
train_no_aug_dataset = CoTDataset(tokenizer, args.train_no_aug_path, 1024)

Creating features from dataset file at /workspace-SR006.nfs2/Bulatov_A/rmt/tools/implicit_chain_of_thought/data/gsm8k/train_no_aug.txt
tgt_avg:  68.31206080596587
src_avg:  55.93819016205364
ratios:  0.8188625771507747
tgt_avg:  6.091352359099384
src_avg:  55.93819016205364
ratios:  9.18321365509123
 The town’s annual budget totals $32 million. If half of the budget goes towards policing and $12 million goes towards education. How much money is left for managing public spaces? <|endoftext|> The annual budget for policing is 32 / 2 = $16 million. The combined budget for education and policing is 16 + 12 = $28 million. There is 32 - 28 = $4 million to manage public spaces. <|endoftext|> #### 4 <|endoftext|>
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 383, 5079, 4466, 329, 21922, 318, 3933, 1220

In [ ]:
# print out first all features of sample
i += 1
for k, v in hf_valid_dataset[i].items():
    tokens = [i if i > 0 else 0 for i in v]
    print(k, tokenizer.decode(tokens))
    print()

examples_cot  Jason has to drive home which is 120 miles away. If he drives at 60 miles per hour for 30 minutes, what speed does he have to average for the remainder of the drive to get there in exactly 1 hour 30 minutes? <|endoftext|> <<60*0.5=30>> <<120-30=90>> <<1.5-0.5=1>> <|endoftext|>

examples_nocot  Jason has to drive home which is 120 miles away. If he drives at 60 miles per hour for 30 minutes, what speed does he have to average for the remainder of the drive to get there in exactly 1 hour 30 minutes? <|endoftext|> #### 90 <|endoftext|>

labels_cot !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! <<60*0.5=30>> <<120-30=90>> <<1.5-0.5=1>> <|endoftext|>

labels_cot_shift !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<|endoftext|> <<60*0.5=30>> <<120-30=90>> <<1.5-0.5=1>> <|endoftext|>

labels_nocot !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! #### 90 <|endoftext|>

src_sent_cot  Jason has to drive home which is 120 miles away. If he drives at 60 miles per hour for 30 minut

In [12]:
valid_hf_data_dict = convert_to_hf_format(val_dataset)
hf_valid_dataset = datasets.Dataset.from_dict(valid_hf_data_dict)

test_hf_data_dict = convert_to_hf_format(test_dataset)
hf_test_dataset = datasets.Dataset.from_dict(test_hf_data_dict)

train_hf_data_dict = convert_to_hf_format(train_no_aug_dataset)
hf_train_dataset = datasets.Dataset.from_dict(train_hf_data_dict)

train_no_aug_hf_data_dict = convert_to_hf_format(train_no_aug_dataset)
hf_train_no_aug_dataset = datasets.Dataset.from_dict(train_no_aug_hf_data_dict)

In [13]:

save_path = "/workspace-SR006.nfs2/Bulatov_A/rmt/data/implicit_chain_of_thought/gsm8k/"

hf_valid_dataset.save_to_disk(os.path.join(save_path, "valid"))
hf_test_dataset.save_to_disk(os.path.join(save_path, "test"))
hf_train_no_aug_dataset.save_to_disk(os.path.join(save_path, "train_no_aug"))
hf_train_dataset.save_to_disk(os.path.join(save_path, "train"))

Saving the dataset (1/1 shards): 100%|██████████| 6973/6973 [00:00<00:00, 68710.78 examples/s]


### collate

In [12]:
# dataset_dir = "/workspace-SR006.nfs2/Bulatov_A/rmt/data/implicit_chain_of_thought/4_by_4_mult"

# train_path = os.path.join(dataset_dir, "train")
# valid_path = os.path.join(dataset_dir, "valid")

# hf_train_dataset = datasets.load_from_disk(train_path)
# hf_valid_dataset = datasets.load_from_disk(valid_path)

In [5]:
# hf_valid_dataset.features

In [8]:
batch = [hf_valid_dataset[i] for i in range(10)]

In [9]:
eot_id = 50256

In [4]:
# input_ids = torch.tensor([e["examples_all"] for e in batch])
# attention_mask = torch.tensor([e["attention_mask"] for e in batch])

In [ ]:
dataset_path = "/workspace-SR006.nfs2/Bulatov_A/rmt/data/implicit_chain_of_thought/4_by_4_mult"
train_dataset = datasets.load_from_disk(os.path.join(dataset_path, "train"))
valid_dataset = datasets.load_from_disk(os.path.join(dataset_path, "valid"))
if os.path.exists(os.path.join(dataset_path, "test")):
    test_dataset = datasets.load_from_disk(os.path.join(dataset_path, "test"))
else:
    test_dataset = datasets.load_from_disk(os.path.join(dataset_path, "valid"))

# unite train and valid datsets in a single one
hf_ds = datasets.DatasetDict({
    "train": train_dataset,
    "valid": valid_dataset
})

# add huggingface access token


hf_ds.push_to_hub('booydar/4_by_4_mult', token='hf_1234567890abcdefg')

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.12it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/booydar/4_by_4_mult/commit/e91a3ffd2668499d817d8c2f7fc3d45adbee3dc8', commit_message='Upload dataset', commit_description='', oid='e91a3ffd2668499d817d8c2f7fc3d45adbee3dc8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/booydar/4_by_4_mult', endpoint='https://huggingface.co', repo_type='dataset', repo_id='booydar/4_by_4_mult'), pr_revision=None, pr_num=None)

In [ ]:
dataset_path = "/workspace-SR006.nfs2/Bulatov_A/rmt/data/implicit_chain_of_thought/gsm8k"
train_dataset = datasets.load_from_disk(os.path.join(dataset_path, "train"))
valid_dataset = datasets.load_from_disk(os.path.join(dataset_path, "valid"))
test_dataset = datasets.load_from_disk(os.path.join(dataset_path, "test"))
train_no_aug_dataset = datasets.load_from_disk(os.path.join(dataset_path, "train_no_aug"))


# unite train and valid datsets in a single one
hf_ds = datasets.DatasetDict({
    "train": train_dataset,
    "valid": valid_dataset,
    "test": test_dataset,
    "test_no_aug": train_no_aug_dataset,
})

# add huggingface access token

hf_ds.push_to_hub('booydar/gsm8k', token='hf_1234567890abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz1234567890!@#$%^&*()_+{}|:"<>?`~АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя1234567890!@#$%^&*()_+{}|:"<>?`~')

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.26s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/booydar/gsm8k/commit/4bce75ba7d3f7df96e5db3ce0446a69e0bb1055c', commit_message='Upload dataset', commit_description='', oid='4bce75ba7d3f7df96e5db3ce0446a69e0bb1055c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/booydar/gsm8k', endpoint='https://huggingface.co', repo_type='dataset', repo_id='booydar/gsm8k'), pr_revision=None, pr_num=None)

In [ ]:
# add huggingface access token


In [10]:
from torch.nn.utils.rnn import pad_sequence

In [11]:
args.num_mem_tokens = None
# args.training_mode = 'no_cot'
args.use_cot = False
if args.use_cot in (False, None):
    inputs_key = 'examples_nocot'
    labels_key = 'labels_nocot'
else:
    inputs_key = 'examples_all'
    labels_key = 'labels_all'

In [12]:
id_pad_value = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
# id_pad_value = -100


if args.use_cot in (False, None):
    inputs_key = 'examples_nocot'
    labels_key = 'labels_nocot'
else:
    inputs_key = 'examples_all'
    labels_key = 'labels_all'
    
def collate_fn(batch):
    input_ids = [torch.tensor(b[inputs_key]) for b in batch]
    labels = [torch.tensor(b[labels_key]) for b in batch]
    attention_mask = [torch.ones_like(b, dtype=int) for b in input_ids]
    # labels_mask defines which input_ids participate in loss calculation
    labels_mask = [torch.sign(torch.tensor(b[labels_key])) for b in batch]


    input_ids = pad_sequence(input_ids, padding_value=id_pad_value, batch_first=True)
    labels = pad_sequence(labels, padding_value=id_pad_value, batch_first=True)
    attention_mask = pad_sequence(attention_mask, padding_value=0, batch_first=True)
    labels_mask = pad_sequence(labels_mask, padding_value=0, batch_first=True)

    collated = {'input_ids': input_ids,
                'labels': labels, 
                'attention_mask': attention_mask,
                }
    if args.num_mem_tokens is not None:
        # add labels mask only for RMT, ARMT
        collated['labels_mask'] = labels_mask.bool()
    return collated

In [13]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained('gpt2')

In [15]:
batch = [valid_dataset[i] for i in range(10)]
collated = collate_fn(batch)

In [16]:
out = model(**collated)

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


In [17]:
collated.keys()

dict_keys(['input_ids', 'labels', 'attention_mask'])

In [18]:
out.loss


tensor(5.7507, grad_fn=<NllLossBackward0>)

### recurrent

In [ ]:
args.use_cot = True

In [ ]:
id_pad_value = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
# id_pad_value = -100


if args.use_cot in (False, None):
    inputs_key = 'examples_nocot'
    labels_key = 'labels_nocot'
else:
    inputs_key = 'examples_all'
    labels_key = 'labels_all'
    
split_cot_by = ">> <<"
# def collate_fn(batch):
input_ids = [torch.tensor(b[inputs_key]) for b in batch]
labels = [torch.tensor(b[labels_key]) for b in batch]
attention_mask = [torch.ones_like(b, dtype=int) for b in input_ids]
# labels_mask defines which input_ids participate in loss calculation
labels_mask = [torch.sign(torch.tensor(b[labels_key])) for b in batch]


input_ids = pad_sequence(input_ids, padding_value=id_pad_value, batch_first=True)
labels = pad_sequence(labels, padding_value=id_pad_value, batch_first=True)
attention_mask = pad_sequence(attention_mask, padding_value=0, batch_first=True)
labels_mask = pad_sequence(labels_mask, padding_value=0, batch_first=True)

collated = {'input_ids': input_ids,
            'labels': labels, 
            'attention_mask': attention_mask,
            }
if args.num_mem_tokens is not None:
    # add labels mask only for RMT, ARMT
    collated['labels_mask'] = labels_mask.bool()
# return collated

In [36]:
# print out first all features of sample

for k, v in valid_dataset[0].items():
    tokens = [i if i > 0 else 0 for i in v]
    print(k, tokenizer.decode(tokens))
    print()

examples_cot  John cuts his grass to 2 inches.  It grows .5 inches per month.  When it gets to 4 inches he cuts it back down to 2 inches.  It cost $100 to get his grass cut.  How much does he pay per year? <|endoftext|> <<4-2=2>> <<2/.5=4>> <<12/4=3>> <<100*3=300>> <|endoftext|>

examples_nocot  John cuts his grass to 2 inches.  It grows .5 inches per month.  When it gets to 4 inches he cuts it back down to 2 inches.  It cost $100 to get his grass cut.  How much does he pay per year? <|endoftext|> #### 300 <|endoftext|>

labels_cot !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! <<4-2=2>> <<2/.5=4>> <<12/4=3>> <<100*3=300>> <|endoftext|>

labels_cot_shift !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<|endoftext|> <<4-2=2>> <<2/.5=4>> <<12/4=3>> <<100*3=300>> <|endoftext|>

labels_nocot !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! #### 300 <|endoftext|>

src_sent_cot  John cuts his grass to 2 inches.  It grows .5 inches per month.  When it gets to 4 inches he

In [27]:
#split sample list by eot token
sample = batch[0]['examples_all']
parts = []
current_part = []
for i, token in enumerate(sample):
    if token == eot_id:
        parts.append(current_part)
        current_part = []
    else:
        current_part.append(token)
parts.append(current_part)  # Add the last part


In [29]:
len(parts)

4

In [ ]:
input_ids = [torch.tensor(b['input_ids']) for b in batch]
    if not("labels" in batch[0].keys()):
        labels = [torch.tensor(b['input_ids'].copy()) for b in batch]
    else:a
        labels = [torch.tensor(b['labels']) for b in batch]
    attention_mask = [torch.ones_like(b, dtype=int) for b in input_ids]

In [ ]:
id_pad_value = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
def collate_fn(batch):
    input_ids = [torch.tensor(b['input_ids']) for b in batch]
    if not("labels" in batch[0].keys()):
        labels = [torch.tensor(b['input_ids'].copy()) for b in batch]
    else:a
        labels = [torch.tensor(b['labels']) for b in batch]
    attention_mask = [torch.ones_like(b, dtype=int) for b in input_ids]


    labels_mask = [torch.ones_like(b, dtype=int) for b in input_ids]
    
    if getattr(args, 'loss_from_last_seg_only', False):
        for m in labels_mask:
            m[:-args.segment_size] = False

    if getattr(args, 'no_loss_from_first_segment', False):
        for m in labels_mask:
            m[:args.segment_size] = False

    input_ids = pad_sequence(input_ids, padding_value=id_pad_value, batch_first=True)
    labels = pad_sequence(labels, padding_value=-100, batch_first=True)
    attention_mask = pad_sequence(attention_mask, padding_value=0, batch_first=True)
    labels_mask = pad_sequence(labels_mask, padding_value=0, batch_first=True)

    collated = {'input_ids': input_ids,
                'labels': labels, 
                'attention_mask': attention_mask,
                }
    if args.num_mem_tokens is not None:
        # add labels mask only for RMT, ARMT
        collated['labels_mask'] = labels_mask.bool()
    # TODO: add masking for response only
    return collated

In [14]:
tokenizer.decode(hf_valid_dataset[0]['examples_cot'])

' 5 6 3 2 * 7 4 3 4 <|endoftext|> 5 5 5 6 1 + 0 0 6 4 9 0 ( 5 5 1 1 1 1 ) + 0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 ) + 0 0 0 0 6 4 9 0 <|endoftext|>'

In [18]:
# hf_valid_dataset[0]['labels_cot']

In [42]:
i = 0

In [ ]:
val_dataset[0]

(tensor([  642,   718,   513,   362,  1635,   767,   604,   513,   604,   220,
         50256,   642,   642,   642,   718,   352,  1343,   657,   657,   718,
           604,   860,   657,   357,   642,   642,   352,   352,   352,   352,
          1267,  1343,   657,   657,   642,   860,   657,   767,   657,   357,
           642,   642,   718,   657,   362,   807,   657,  1267,  1343,   657,
           657,   657,   657,   718,   604,   860,   657,   220, 50256]),
 tensor([  642,   718,   513,   362,  1635,   767,   604,   513,   604,   220,
         50256,  1303, 21017,   642,   642,   718,   657,   807,   362,   657,
           352,   220, 50256]),
 tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,   642,   642,   642,   718,   352,  1343,   657,   657,   718,
           604,   860,   657,   357,   642,   642,   352,   352,   352,   352,
          1267,  1343,   657,   657,   642,   860,   657,   767,   657,   357,
           642,   642,   

In [15]:
gen = iter(train_dataloader)
batch = next(gen)

NameError: name 'train_dataloader' is not defined